In [1]:
# ML_in_Finance-1D-CNNs
# Author: Matthew Dixon
# Version: 1.0 (28.4.2020)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X running Python 3.6.9 with the following packages:
# tensorflow=2.0.0, keras=2.3.1, numpy=1.18.1
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Bilokon P., Dixon M.F. and Halperin I., Machine Learning in Finance: From Theory to Practice, Springer Graduate Textbook Series, 2020. 

# Using Keras to implement a 1D convolutional neural network (CNN) for timeseries prediction.

In [2]:
import numpy as np

from keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
np.set_printoptions(threshold=25)

### Creating the 1D CNN

We create a simple convolutional neural network: a 1D convolutional layer, followed by a dense layer.

It will allow us to predict the next value in a timeseries given an input sequence of length `window_size`

The `filter_length` is the length (in time-steps) of the sliding window that gets convolved with each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
`nb_filter` is the number of such filters to learn (roughly, input patterns to recognize).

The model can handle multivariate timeseries (with `nb_input_series` variables) and multiple (`nb_outputs`) prediction targets. Predicting future values of a timeseries means setting these equal to one another.

In [4]:
def make_CNN(window_size, filter_length,  nb_filter=4, nb_input_series=1, nb_outputs=1):
    """
    window_size (int): number of observations in each input sequence
    filter length (int): length of the convolutional layer's filters
    nb_filter (int): number of filters learned in the convolutional layer
    nb_input_series (int): number of features of the input timeseries (1 for a univariate timeseries)
    nb_outputs (int): number of features being predicted (equal to nb_input_series 
        for predicting a timeseries at a set horizon)
    """
    model = Sequential((
        # The convolutional layer learns `nb_filter` filters (aka kernels), 
        # each of size `(filter_length, nb_input_series)`.  
        # Its output will have shape `(None, window_size - filter_length + 1, nb_filter)` ,  
        # i.e., for each position in the input timeseries, the activation of each filter at that position.
        Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)),
        Flatten(),
        Dense(nb_outputs, activation='linear'), # For classification, a 'sigmoid' activation function would be used
    ))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    return model

In [6]:
CNN_model = make_CNN(window_size=50, filter_length=5, nb_filter=4)

In [7]:
print('input shape:', CNN_model.layers[0].input_shape)
print('output shape:', CNN_model.layers[-1].output_shape)

input shape: (None, 50, 1)
output shape: (None, 1)


### Data preparation
  
We define a function to format a timeseries for training the neural network. It creates corresponding arrays of input sequences `X` and output values `y`. They have the same length as each other; the remaining dimensions must match the input and output layers of the model respectively:

The `X` input to the model's `fit()` method should be a 3D array of shape `(n_instances, window_size, n_ts_variables)`; each instance being a 2D array of shape `(window_size, nb_input_series)`.  For example, for `window_size = 3` and `nb_input_series = 1` (a univariate timeseries), one instance could be `[[0], [1], [2]]`

For each input instance, the output is a vector of size `nb_outputs`, usually the value(s) predicted to come after the last value in that input instance, i.e., the next value in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``. 


In [8]:
def make_timeseries_instances(timeseries, window_size):
    # Convert 1D vectors to 2D column vectors
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T 
    
    if not 0 < window_size < timeseries.shape[0]:
        raise ValueError('Please set 0 < window size < timeseries length')
    
    # `X `is the tensor containing the inputs for the model
    # each row of `X` is a sequence of `window_size` observations from the timeseries
    X = [timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]
    
    # for training the model, the array's dimensions must match the input layer of the CNN
    # that is, a 3D array of shape (timeseries.shape[0] - window_size, window_size, nof_ts_variables)
    X = np.atleast_3d(np.array(X))
    
    # For each row of `X`, the corresponding row of `y` is the 
    # desired output -- in this case, the subsequent value in the timeseries 
    y = timeseries[window_size:]
    
    return X, y

For example:  

In [9]:
X_fib, y_fib = make_timeseries_instances([1,1,2,3,5,8,13,21], 5)
print('X:', X_fib, 'y:', y_fib, sep='\n')

X:
[[[ 1]
  [ 1]
  [ 2]
  [ 3]
  [ 5]]

 [[ 1]
  [ 2]
  [ 3]
  [ 5]
  [ 8]]

 [[ 2]
  [ 3]
  [ 5]
  [ 8]
  [13]]]
y:
[[ 8]
 [13]
 [21]]


Create a toy timeseries and split it for training and testing the CNN:

In [10]:
timeseries = np.arange(1000)

In [11]:
X, y = make_timeseries_instances(timeseries, window_size=50)

In [12]:
i = 42
print('input instance:\n', X[i])
print('output instance:\n', y[i])

input instance:
 [[42]
 [43]
 [44]
 ...
 [89]
 [90]
 [91]]
output instance:
 [92]


In [13]:
test_ratio = 0.01 # In real life you'd usually want to use 0.2 - 0.5
test_size = int(test_ratio * len(timeseries)) 

# the "most recent" values are used for testing the model to avoid look-ahead bias
X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]

Note the dimensions of the arrays:

In [14]:
[i.shape for i in [X_train, X_test, y_train, y_test]]

[(940, 50, 1), (10, 50, 1), (940, 1), (10, 1)]

### Training the model

Now we can fit the model. Note that `validation_data` is not used to train the model, but allows you to monitor its out-of-sample performance during training

In [15]:
CNN_model.fit(X_train, y_train, epochs=25, batch_size=2, validation_data=(X_test, y_test))

Train on 940 samples, validate on 10 samples
Epoch 1/25
940/940 [==============================] - 8s 8ms/step - loss: 22348.4393 - mae: 55.2219 - val_loss: 346.1044 - val_mae: 18.6034
Epoch 2/25
940/940 [==============================] - 4s 4ms/step - loss: 197.2894 - mae: 11.7657 - val_loss: 155.3413 - val_mae: 12.4630
Epoch 3/25
940/940 [==============================] - 4s 5ms/step - loss: 198.5208 - mae: 11.8470 - val_loss: 802.1042 - val_mae: 28.3210
Epoch 4/25
940/940 [==============================] - 4s 4ms/step - loss: 180.0273 - mae: 11.2627 - val_loss: 280.3401 - val_mae: 16.7429
Epoch 5/25
940/940 [==============================] - 4s 5ms/step - loss: 176.2927 - mae: 11.1332 - val_loss: 557.3866 - val_mae: 23.6086
Epoch 6/25
940/940 [==============================] - 4s 5ms/step - loss: 154.2343 - mae: 10.4821 - val_loss: 297.4342 - val_mae: 17.2459
Epoch 7/25
940/940 [==============================] - 4s 4ms/step - loss: 142.0517 - mae: 9.9618 - val_loss: 162.1052 - val_m

We can inspect the weights of the convolutional layer:

In [26]:
CNN_model.summary()

CNN_model.get_layer('conv1d_1').weights

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 46, 4)             24        
_________________________________________________________________
flatten_1 (Flatten)          (None, 184)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 185       
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'conv1d_1/kernel:0' shape=(5, 1, 4) dtype=float32, numpy=
 array([[[-0.34088135, -0.07874674,  0.4062053 ,  0.41992363]],
 
        [[-0.00244047, -0.03935403,  0.29828435, -0.25241745]],
 
        [[-0.22607194, -0.03523868, -0.3957395 , -0.41727823]],
 
        [[ 0.3158957 ,  0.2066541 ,  0.39682862, -0.3310415 ]],
 
        [[-0.13993861, -0.4061464 , -0.19353136,  0.14599642]]],
       dtype=float32)>,
 <tf.Variable 'conv1d_1/bias:0' shape=(4,) dtype=float32, numpy=array([-0.05061084,  0.        ,  2.90622   ,  0.        ], dtype=float32)>]

### Making predictions with the model
Get the predicted values for the test set:

In [17]:
y_pred = CNN_model.predict(X_test)

In [18]:
print(np.column_stack((y_test, y_pred)))

[[ 990.          991.97485352]
 [ 991.          992.97735596]
 [ 992.          993.97973633]
 [ 993.          994.98217773]
 [ 994.          995.98461914]
 [ 995.          996.98706055]
 [ 996.          997.98950195]
 [ 997.          998.99188232]
 [ 998.          999.99438477]
 [ 999.         1000.99682617]]


### Multiple-input, multiple-output prediction

In [19]:
n_vars = 4
mv_timeseries = np.array([i * np.arange(1000) for i in np.arange(1, n_vars + 1)]).T
print(mv_timeseries)

[[   0    0    0    0]
 [   1    2    3    4]
 [   2    4    6    8]
 ...
 [ 997 1994 2991 3988]
 [ 998 1996 2994 3992]
 [ 999 1998 2997 3996]]


In [20]:
mv_X, mv_y = make_timeseries_instances(mv_timeseries, 50)

print('X[0]: ', mv_X[0], 'y[0]:', mv_y[0], sep='\n')

X[0]: 
[[  0   0   0   0]
 [  1   2   3   4]
 [  2   4   6   8]
 ...
 [ 47  94 141 188]
 [ 48  96 144 192]
 [ 49  98 147 196]]
y[0]:
[ 50 100 150 200]


In [21]:
test_ratio = 0.01 # In real life you'd usually want to use 0.2 - 0.5
test_size = int(test_ratio * len(timeseries)) 

mv_X_train, mv_X_test = mv_X[:-test_size], mv_X[-test_size:] 
mv_y_train, mv_y_test = mv_y[:-test_size], mv_y[-test_size:]

In [22]:
mv_CNN_model = make_CNN(window_size=50, filter_length=5, nb_filter=4, nb_input_series=n_vars, nb_outputs=n_vars)

In [23]:
mv_CNN_model.fit(mv_X_train, mv_y_train, epochs=25, batch_size=2, validation_data=(mv_X_test, mv_y_test))

Train on 940 samples, validate on 10 samples
Epoch 1/25
940/940 [==============================] - 9s 10ms/step - loss: 67570.3614 - mae: 70.6544 - val_loss: 429.2058 - val_mae: 18.6915
Epoch 2/25
940/940 [==============================] - 4s 4ms/step - loss: 570.9748 - mae: 17.0400 - val_loss: 1157.8059 - val_mae: 29.1417
Epoch 3/25
940/940 [==============================] - 4s 5ms/step - loss: 651.1784 - mae: 18.1105 - val_loss: 379.6758 - val_mae: 17.3287
Epoch 4/25
940/940 [==============================] - 5s 5ms/step - loss: 567.5281 - mae: 17.1644 - val_loss: 2001.1688 - val_mae: 34.8229
Epoch 5/25
940/940 [==============================] - 4s 4ms/step - loss: 596.8147 - mae: 17.6290 - val_loss: 964.9664 - val_mae: 27.7559
Epoch 6/25
940/940 [==============================] - 4s 4ms/step - loss: 542.8998 - mae: 16.8306 - val_loss: 166.5247 - val_mae: 10.1677
Epoch 7/25
940/940 [==============================] - 4s 4ms/step - loss: 413.6827 - mae: 14.8041 - val_loss: 2493.7627 - 

In [24]:
mv_y_pred = mv_CNN_model.predict(mv_X_test)

In [25]:
for i in range(test_size):
    print('true:', mv_y_test[i], '\tpred:', mv_y_pred[i])

true: [ 990 1980 2970 3960] 	pred: [ 980.3531 1968.6608 2958.1584 3950.858 ]
true: [ 991 1982 2973 3964] 	pred: [ 981.3445 1970.6503 2961.1492 3954.8552]
true: [ 992 1984 2976 3968] 	pred: [ 982.3357 1972.6409 2964.1401 3958.8518]
true: [ 993 1986 2979 3972] 	pred: [ 983.32697 1974.6309  2967.1316  3962.8486 ]
true: [ 994 1988 2982 3976] 	pred: [ 984.31824 1976.621   2970.1228  3966.8457 ]
true: [ 995 1990 2985 3980] 	pred: [ 985.3092 1978.6108 2973.1133 3970.8428]
true: [ 996 1992 2988 3984] 	pred: [ 986.3005 1980.6011 2976.1047 3974.8394]
true: [ 997 1994 2991 3988] 	pred: [ 987.2918 1982.591  2979.0962 3978.836 ]
true: [ 998 1996 2994 3992] 	pred: [ 988.28284 1984.5813  2982.0872  3982.833  ]
true: [ 999 1998 2997 3996] 	pred: [ 989.274  1986.5712 2985.0784 3986.8303]
